In [7]:
import pandas as pd
import numpy as np
import codecs
import math
import copy
import random

In [2]:
with codecs.open("data/countries.csv", "r", "utf-8", "ignore") as file:
    df = pd.read_table(file,encoding = "utf-8", delimiter=",")

# remove UM because it has no latitude or longitude
df = df[df['country']!='UM']
df = df.dropna(how='any')
df

,country,latitude,longitude,name
0,AD,42.546245,1.601554,Andorra
1,AE,23.424076,53.847818,United Arab Emirates
2,AF,33.939110,67.709953,Afghanistan
3,AG,17.060816,-61.796428,Antigua and Barbuda
4,AI,18.220554,-63.068615,Anguilla
...,...,...,...,...
240,YE,15.552727,48.516388,Yemen
241,YT,-12.827500,45.166244,Mayotte
242,ZA,-30.559482,22.937506,South Africa
243,ZM,-13.133897,27.849332,Zambia


In [3]:
# calculate distance
def calc_distance(latitudeA, longitudeA, latitudeB, longitudeB, ):
    rlaA = math.radians(latitudeA)
    rloA = math.radians(longitudeA)
    
    rlaB = math.radians(latitudeB)
    rloB = math.radians(longitudeB)

    # 北極からの角度
    a = math.pi/2-rlaA
    b = math.pi/2-rlaB
    # 経度の差
    C = rloA - rloB

    return math.acos(max(min(math.cos(a) * math.cos(b) + math.sin(a) * math.sin(b) * math.cos(C), 1), -1))

In [4]:
country2lola = {}
for country, latitude, longitude in zip(list(df['country']), list(df['latitude']), list(df['longitude'])):
    country2lola[country] = (latitude, longitude)

In [5]:
distance_matrix = {}
for country_i in list(country2lola.keys()):
    distance_matrix_i = {}
    for country_j in list(country2lola.keys()):
        distance_matrix_i[country_j] = calc_distance(country2lola[country_i][0], country2lola[country_i][1], country2lola[country_j][0], country2lola[country_j][1])
    distance_matrix[country_i] = distance_matrix_i

In [49]:
class JOURNEY():
    route = []
    Ndestinations = 0
    def __init__(self, route):
        self.route         = route
        self.Ndestinations = len(route)
    def get_similar_route(self, distance_matrix): # return {route: distance}
        route2distance = {}
        self_distance  = self.get_total_distance(distance_matrix)
        for i in list(range(self.Ndestinations-1)):
            for j in list(range(self.Ndestinations))[i+1:]:
                #j = i + 1
                #if True:                                    # 出発点と目的点も変える
                if (i != 0 and j != self.Ndestinations - 1): # 出発点と目的点は変えない
                    route_        = copy.deepcopy(self.route)
                    # swap i-th and j-th destinations
                    tmp       = route_[i]
                    route_[i] = route_[j]
                    route_[j] = tmp
                    # calculate total distance
                    similar_journey = JOURNEY(route_)
                    route2distance[similar_journey] = similar_journey.get_total_distance(distance_matrix)
        return route2distance
    def get_total_distance(self, distance_matrix):
        total_distance = 0
        for i in range(self.Ndestinations-1):
            total_distance += distance_matrix[self.route[i]][self.route[i+1]]
        return total_distance

In [61]:
def optimize_journey(route):
    journey = JOURNEY(route)

    T = 10000 # temperature

    while T > 0:
        journey2distance = journey.get_similar_route(distance_matrix)
        min_journey = min(journey2distance, key=journey2distance.get)

        score_difference = min_journey.get_total_distance(distance_matrix) - journey.get_total_distance(distance_matrix)

        print(score_difference, np.exp(-(score_difference)/T), end = '\t')

        if (score_difference < 0):
            print(journey.get_total_distance(distance_matrix))
            journey = min_journey
        elif random.random() < np.exp(-(score_difference)/T):
            print(journey.get_total_distance(distance_matrix))
            journey = min_journey
        T -= 1
    return journey.route

In [62]:
route = list(df['country'])
min_journey = JOURNEY(route)
for i in range(100):
    random.shuffle(route)
    journey = JOURNEY(route)
    if journey.get_total_distance(distance_matrix) < min_journey.get_total_distance(distance_matrix):
        min_journey = journey

In [63]:
journey.get_total_distance(distance_matrix)

324.0272155262677

In [60]:
optimized_journey = optimize_journey(journey.route)

9.117956294903308 0.9990886199298333	333.7477147381238
7.423174245478776 0.9992578838407953	324.6297584432205
7.345290457162378 0.9992655938274081	317.20658419774173
6.754545200709117 0.9993245709877026	309.86129374057936
6.484993166431536 0.9993514515783304	303.10674853987024
7.2881638362067065 0.9992710848137324	296.6217553734387
6.3879289329936455 0.9993610278305805	289.333591537232
6.003170326259067 0.9993994428577637	282.94566260423835
5.635855842600847 0.9994361222251499	276.9424922779793
5.37634568755692 0.9994620258842511	271.30663643537844
5.104365529627785 0.9994891830109818	265.9302907478215
5.6884059248060055 0.9994306951093735	260.82592521819373
5.068687021861308 0.9994926510696196	255.13751929338773
5.028851244989966 0.9994965870287087	250.06883227152642
4.898222361995067 0.9995096113311197	245.03998102653645
5.063043797234741 0.9994930635585301	240.14175866454138
4.8944995532972655 0.9995098858149573	235.07871486730664
4.823069004054219 0.9995169884692686	230.18421531400

KeyboardInterrupt: 